In [21]:
import cohere
import os 
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from pprint import pprint
from dotenv import load_dotenv

from openai import OpenAI

In [3]:
client = OpenAI()
HF_EMBEDDING = HuggingFaceEmbeddings(model_name="keepitreal/vietnamese-sbert")
vectorstore = Chroma(persist_directory="./data" ,embedding_function=HF_EMBEDDING)
retriever =vectorstore.as_retriever()

/home/tranhuy/Desktop/Project/cantho-public-services/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
# Reciprocal Rank Fusion algorithm
def reciprocal_rank_fusion(search_results_dict, k=60):
    fused_scores = {}        
    for query, doc_scores in search_results_dict.items():
        for rank, (doc, score) in enumerate(sorted(doc_scores.items(), key=lambda x: x[1], reverse=True)):
            if doc not in fused_scores:
                fused_scores[doc] = 0
            fused_scores[doc] += 1 / (rank + k)
            # print(f"Updating score for {doc} from {previous_score} to {fused_scores[doc]} based on rank {rank} in query '{query}'")

    reranked_results = {doc: score for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)}
    # print("Final reranked results:", reranked_results)
    return reranked_results

def generating_search_results(question):
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "Bạn là một chatbot hỗ trợ sinh các câu hỏi đồng nghĩa dựa vào câu hỏi gốc. Hãy sinh ra 3 câu hỏi, phân cách bằng ký tự '\n'"},
            {"role": "system", "content": question},
        ]
    )
    questions = completion.choices[0].message.content.split('\n')
    # pprint(questions)
    search_results_dict = {}
    for question in questions:
        scored_docs = vectorstore.similarity_search_with_relevance_scores(question)
        result = {doc.page_content: score for (doc, score) in scored_docs}
        search_results_dict[question] = result

    return search_results_dict

In [15]:
question = "Muốn chuyển nhượng quyền sở hữu đất cho con cần những giấy tờ gì"
search_results_dict = generating_search_results(question)
# pprint(search_results_dict)
reranked_results = reciprocal_rank_fusion(search_results_dict)
print("Final number of documents: ", len(reranked_results))
pprint(reranked_results)

{'Cơ quan thực hiện, ủy quyền, thẩm quyền, điều kiện thực hiện, kết quả thực hiện của thủ tục hành chính Đăng ký biến động quyền sử dụng đất, quyền sở hữu tài sản gắn liền với đất trong các trường hợp chuyển đổi, chuyển nhượng, cho thuê, cho thuê lại, thừa kế, tặng cho, góp vốn bằng quyền sử dụng đất, quyền sở hữu tài sản gắn liền với đất; chuyển quyền sử dụng đất, quyền sở hữu tài sản gắn liền với đất của vợ hoặc chồng thành của chung vợ và chồng; tăng thêm diện tích do nhận chuyển nhượng, thừa kế, tặng cho quyền sử dụng đất đã có Giấy chứng nhận (cấp tỉnh -trường hợp đã thành lập VP đăng ký đất đai) như sau:\n- Cơ quan thực hiện: Văn phòng đăng ký đất đai, Chi nhánh Văn phòng đăng ký đất đai\n- Cơ quan thẩm quyền: Sở Tài nguyên và Môi trường\n- Cơ quan ủy quyền: Không có thông tin\n- Cơ quan phối hợp: Cơ quan thuế\nĐể thực hiện thủ tục hành chính Đăng ký biến động quyền sử dụng đất, quyền sở hữu tài sản gắn liền với đất trong các trường hợp chuyển đổi, chuyển nhượng, cho thuê, cho th

/home/tranhuy/Desktop/Project/cantho-public-services/.venv/lib/python3.10/site-packages/langchain_core/vectorstores.py:313: UserWarning: Relevance scores must be between 0 and 1, got [(Document(page_content='Cần chuẩn bị các hồ sơ sau đây và nộp tại Không có thông tin với thủ tục hành chính Đăng ký, cấp Giấy chứng nhận quyền sử dụng đất, quyền sở hữu nhà ở và tài sản khác gắn liền với đất đối với trường hợp đã chuyển quyền sử dụng đất trước ngày 01 tháng 7 năm 2014 mà bên chuyển quyền đã được cấp Giấy chứng nhận nhưng chưa thực hiện thủ tục chuyển quyền theo quy định (đã thành lập):\n- Ghi chú: - Trường hợp nhận chuyển nhượng, nhận tặng cho quyền sử dụng đất nhưng không lập hợp đồng, văn bản chuyển quyền theo quy định, hồ sơ gồm có:\n2. + Bản gốc Giấy chứng nhận đã cấp;: cần 1 bản chính và 0 bản sao.. Mẫu đơn cho hồ sơ này có thể tìm thấy tại \n3. + Đơn đăng ký biến động đất đai, tài sản gắn liền với đất theo Mẫu số 09/ĐK;: cần 1 bản chính và 0 bản sao.. Mẫu đơn cho hồ sơ này có thể tì

In [23]:
load_dotenv()
COHERE_API_KEY = os.getenv("COHERE_API_KEY")
co = cohere.Client(api_key=COHERE_API_KEY)
docs = list(reranked_results.keys())
results = co.rerank(model="rerank-multilingual-v2.0", query=question, documents=docs, top_n=3)
results

[RerankResult<document['text']: Cần chuẩn bị các hồ sơ sau đây và nộp tại Không có thông tin với thủ tục hành chính Thủ tục chuyển mục đích sử dụng đất phải được phép của cơ quan nhà nước có thẩm quyền đối với hộ gia đình, cá nhân:
1. (1) Đơn xin phép chuyển mục đích sử dụng đất.: cần 1 bản chính và 0 bản sao.. Mẫu đơn cho hồ sơ này có thể tìm thấy tại 
2. (2) Giấy chứng nhận quyền sử dụng đất hoặc Giấy chứng nhận quyền sở hữu nhà ở và quyền sử dụng đất ở hoặc Giấy chứng nhận quyền sử dụng đất, quyền sở hữu nhà ở và tài sản khác gắn liền với đất.: cần 1 bản chính và 0 bản sao.. Mẫu đơn cho hồ sơ này có thể tìm thấy tại, index: 2, relevance_score: 0.966028>, RerankResult<document['text']: Cần chuẩn bị các hồ sơ sau đây và nộp tại Không có thông tin với thủ tục hành chính Đăng ký, cấp Giấy chứng nhận quyền sử dụng đất, quyền sở hữu nhà ở và tài sản khác gắn liền với đất đối với trường hợp đã chuyển quyền sử dụng đất trước ngày 01 tháng 7 năm 2014 mà bên chuyển quyền đã được cấp Giấy chứn